# Algoritmo final: Decision Tree

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import datetime

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
inicio = datetime.datetime.now()

In [130]:
propiedades = pd.read_csv('../../set_datos_sup_desc.csv')
set_pruebas = pd.read_csv("../properati_dataset_modificado_descripciones.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [131]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades.loc[:, 'Year'] = propiedades.loc[:,'created_on'].apply(lambda x: int(x.split('-')[0]))

propiedades.loc[:, 'Month'] = propiedades.loc[:,'created_on'].apply(lambda x: int(x.split('-')[1]))


propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull())\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min) &\
                              (propiedades.Year >=2017),:]


# Columnas

In [132]:
col = ['superficie','lat', 'lon','property_type_encoded']

comodidades = ['seguridad','gimnasio', 'aire', 'pileta', 'cochera','transporte','cocina','comedor',\
'living', 'banos', 'balcon','ambientes','departamento', 'dormitorios','quincho','porcelanato',\
'completo', 'edificio','lavadero', 'amplio', 'planta', 'cuadras', 'placard', 'terraza', 'acceso',\
'parrilla', 'luminoso','madera', 'credito', 'calidad', 'cucicba', 'amenities','expensas','mesada',\
'responsable','muebles', 'diario', 'independiente', 'cubierta','contrafrente', 'integrada',\
'recepcion','categoria', 'profesional', 'baulera', 'natural','unidades', 'principal', 'espacio', 'estacion', 'vestidor',\
'solarium', 'central','parquet', 'estrenar', 'granito','patio',  'casa', 'suite', 'vista', 'hall', 'agua',  \
'lote', 'fondo', 'acero', 'torre',  'horno',  'sala', 'salon', 'frente','toilette','piso', 'pisos', 'apto',\
'sobre','doble','marca','nuevo', 'bacha', 'ideal','excelente']

len(comodidades)

81

# Parametros del estimador

In [133]:
criterion = 'mse'
max_depth = 20
max_features = 0.8

# Con un set de validacion

In [117]:
columnas = col + comodidades
X_train, X_test, y_train, y_test = train_test_split(propiedades.loc[:,columnas+['description']], \
                                        propiedades.loc[:,'price_aprox_usd'], test_size=0.2)
X_train = pd.DataFrame(X_train).reset_index(drop = True)
X_test = pd.DataFrame(X_test).reset_index(drop = True)

In [118]:
tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features)
    
tree.fit(X_train.loc[:,columnas],y_train)
a = tree.predict(X_test.loc[:,columnas])
precision = tree.score(X_test.loc[:,columnas],y_test) * 100
error = mean_squared_error(tree.predict(X_test.loc[:,columnas]), y_test)
print("Precision = {:.2f} %,  Error = {:.0f}".format(precision,error))

Precision = 93.49 %,  Error = 2487568555


In [58]:
#pd.DataFrame(tree.feature_importances_, columnas).sort_values(by= 0,ascending = False)

In [29]:
#pd.DataFrame(X_test).iloc[27014]

In [55]:
#propiedades.description

In [119]:
b = pd.DataFrame(a,y_test).reset_index()
b.loc[:, 'd'] = X_test.description
b.columns = ['real','pred', 'd']
b['dif'] = abs(b.pred - b.real)
b.sort_values(by = 'dif', ascending = False)


,real,pred,d,dif
15996,365000.00,3.000000e+06,Corredor Responsable: Franco Savone - CUCICBA ...,2.635000e+06
655,365000.00,3.000000e+06,Corredor Responsable: Franco Savone - CUCICBA ...,2.635000e+06
30762,144900.00,2.300000e+06,Corredor Responsable: Mario Gabriel Gerez - CM...,2.155100e+06
13333,135000.00,2.003000e+06,Hermoso 2 amb. al frente APTO PROFESIONAL!!pis...,1.868000e+06
24448,2100000.00,3.968482e+05,Corredor Responsable: Uno Bienes Raices SRL - ...,1.703152e+06
13959,1668000.00,8.900000e+04,Departamento de dos ambientes - Venta al pozoO...,1.579000e+06
32669,145000.00,1.700000e+06,"Local al frente de 6x10m aprox., con gran terr...",1.555000e+06
27345,1600000.00,2.350000e+05,Corredor Responsable: Uno Bienes Raices SRL - ...,1.365000e+06
7252,1350000.00,6.300000e+04,CGI Impecable dos ambientes al frente con balc...,1.287000e+06
4344,43000.00,9.900000e+05,None- Ancho planta baja: 0- Banos individuales...,9.470000e+05


In [129]:
print(b.d[12675])
#print(b.d[34957])
#print(b.d[35489])
#X_test.property_type_encoded[35489]

Departamento con vista al Rio de la Plata.ESTAS PROPIEDADES SON FALSAS, ESTA PAGINA WEB ES A MODO DE EJEMPLO DEL SISTEMA TOKKO BROKER


In [81]:
propiedades.loc[propiedades.description.str.lower().str.contains('falsa'),'price_aprox_usd']

247240     147000.00
249251     395000.00
304665     149500.00
306908     395000.00
354709     147000.00
356545     395000.00
399877     142000.00
401245     380000.00
409925     239000.00
413062    2916642.36
413063    2916642.36
413064     170000.00
413065    2916642.36
413066    2916642.36
413067    2916642.36
413068    2916642.36
Name: price_aprox_usd, dtype: float64

In [31]:
b.loc[(b.pred - b.real) > 10000,:].count()

pred    2319
real    2319
dif     2319
dtype: int64

# Con el set de pruebas

In [135]:
cant = 50

n = int(len(comodidades) * 0.8)

for i in range(cant):
    com = list(np.random.choice(comodidades,n,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + \
                set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [136]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/AlgoritmoFinal_clase.csv', index = False)

In [11]:
fin = datetime.datetime.now()
tiempo = fin - inicio
print("Tardo {:.0f} minutos.".format(tiempo.seconds / 60))

Tardo 2 minutos.


In [39]:
X_train

,superficie,lat,lon,property_type_encoded,seguridad,gimnasio,aire,pileta,cochera,transporte,...,piso,pisos,apto,sobre,doble,marca,nuevo,bacha,ideal,excelente
0,27.0,-34.545007,-58.459524,1,False,False,False,False,False,False,...,False,False,True,True,False,False,False,False,False,False
1,45.0,-34.745427,-58.395094,1,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,53.0,-34.562569,-58.465833,1,False,False,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
3,55.0,-34.603077,-58.452347,3,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4,190.0,-34.375655,-58.697933,2,False,False,True,True,True,False,...,True,False,False,True,True,False,False,False,False,True
5,34.0,-34.600133,-58.389638,1,False,False,True,False,False,True,...,True,False,True,False,False,False,True,False,False,True
6,120.0,-34.565525,-58.551489,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,50.0,-34.623449,-58.397781,1,False,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,True
8,140.0,-34.549072,-58.464195,2,False,False,True,False,False,True,...,True,True,False,False,False,False,False,False,False,True
9,184.0,-34.561317,-58.443551,1,True,False,False,False,False,False,...,True,False,False,True,True,False,False,False,False,True


In [46]:
propiedades.loc[]

Int64Index([126103, 126104, 126105, 126106, 126107, 126108, 126109, 126110,
            126111, 126112,
            ...
            422923, 422924, 422925, 422926, 422927, 422928, 422929, 422930,
            422931, 422932],
           dtype='int64', length=177448)